# 6강) FAISS 실습

### Requirements

In [ ]:
!pip install datasets
!pip install transformers
!pip install faiss-cpu

     |████████████████████████████████| 8.2MB 8.2MB/s 


## 5강의 자료를 활용해 passage / question enocoder 학습


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
import numpy as np
from tqdm import tqdm, trange
import random
import torch
import torch.nn.functional as F
from transformers import BertModel, BertPreTrainedModel, AdamW, TrainingArguments, get_linear_schedule_with_warmup
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset, SequentialSampler)

torch.manual_seed(3532812018032770127)
torch.cuda.manual_seed(3532812018032770127)
np.random.seed(324)
random.seed(2021)

In [ ]:
dataset = load_dataset("squad_kor_v1")
corpus = list(set([example['context'] for example in dataset['train']]))
len(corpus)

model_checkpoint = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Dataset squad_kor_v1 downloaded and prepared to /root/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/31982418accc53b059af090befa81e68880acc667ca5405d30ce6fa7910950a7. Subsequent calls will reuse this data.


Training Dataset 준비

In [ ]:
# Use subset (10 example) of original training dataset 
sample_idx = np.random.choice(range(len(dataset['train'])), 20)
training_dataset = dataset['train'][sample_idx]

# set number of neagative sample
num_neg = 3

corpus = np.array(corpus)
p_with_neg = []

for c in training_dataset['context']:
  while True:
    neg_idxs = np.random.randint(len(corpus), size=num_neg)

    if not c in corpus[neg_idxs]:
      p_neg = corpus[neg_idxs]

      p_with_neg.append(c)
      p_with_neg.extend(p_neg)
      break

In [ ]:
q_seqs = tokenizer(training_dataset['question'], padding="max_length", truncation=True, return_tensors='pt')
p_seqs = tokenizer(p_with_neg, padding="max_length", truncation=True, return_tensors='pt')

max_len = p_seqs['input_ids'].size(-1)
p_seqs['input_ids'] = p_seqs['input_ids'].view(-1, num_neg+1, max_len)
p_seqs['attention_mask'] = p_seqs['attention_mask'].view(-1, num_neg+1, max_len)
p_seqs['token_type_ids'] = p_seqs['token_type_ids'].view(-1, num_neg+1, max_len)

train_dataset = TensorDataset(p_seqs['input_ids'], p_seqs['attention_mask'], p_seqs['token_type_ids'], 
                        q_seqs['input_ids'], q_seqs['attention_mask'], q_seqs['token_type_ids'])

BERT encoder 학습시키기

In [ ]:
class BertEncoder(BertPreTrainedModel):
  def __init__(self, config):
    super(BertEncoder, self).__init__(config)

    self.bert = BertModel(config)
    self.init_weights()
      
  def forward(self, input_ids, 
              attention_mask=None, token_type_ids=None): 
  
      outputs = self.bert(input_ids,
                          attention_mask=attention_mask,
                          token_type_ids=token_type_ids)
      
      pooled_output = outputs[1]

      return pooled_output


def train(args, num_neg, dataset, p_model, q_model):
  
  # Dataloader
  train_sampler = RandomSampler(dataset)
  train_dataloader = DataLoader(dataset, sampler=train_sampler, batch_size=args.per_device_train_batch_size)

  # Optimizer
  no_decay = ['bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [
        {'params': [p for n, p in p_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
        {'params': [p for n, p in p_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
        {'params': [p for n, p in q_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
        {'params': [p for n, p in q_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
  optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
  t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total)

  # Start training!
  global_step = 0
  
  p_model.zero_grad()
  q_model.zero_grad()
  torch.cuda.empty_cache()
  
  train_iterator = trange(int(args.num_train_epochs), desc="Epoch")

  for _ in train_iterator:
    epoch_iterator = tqdm(train_dataloader, desc="Iteration")

    for step, batch in enumerate(epoch_iterator):
      q_encoder.train()
      p_encoder.train()
      
      targets = torch.zeros(args.per_device_train_batch_size).long()
      if torch.cuda.is_available():
        batch = tuple(t.cuda() for t in batch)
        targets = targets.cuda()

      p_inputs = {'input_ids': batch[0].view(
                                    args.per_device_train_batch_size*(num_neg+1), -1),
                  'attention_mask': batch[1].view(
                                    args.per_device_train_batch_size*(num_neg+1), -1),
                  'token_type_ids': batch[2].view(
                                    args.per_device_train_batch_size*(num_neg+1), -1)
                  }
      
      q_inputs = {'input_ids': batch[3],
                  'attention_mask': batch[4],
                  'token_type_ids': batch[5]}
      
      p_outputs = p_model(**p_inputs)  #(batch_size*(num_neg+1), emb_dim)
      q_outputs = q_model(**q_inputs)  #(batch_size*, emb_dim)

      # Calculate similarity score & loss
      p_outputs = p_outputs.view(args.per_device_train_batch_size, -1, num_neg+1)
      q_outputs = q_outputs.view(args.per_device_train_batch_size, 1, -1)

      sim_scores = torch.bmm(q_outputs, p_outputs).squeeze()  #(batch_size, num_neg+1)
      sim_scores = sim_scores.view(args.per_device_train_batch_size, -1)
      sim_scores = F.log_softmax(sim_scores, dim=1)

      loss = F.nll_loss(sim_scores, targets)
      print(loss)

      loss.backward()
      optimizer.step()
      scheduler.step()
      q_model.zero_grad()
      p_model.zero_grad()
      global_step += 1
      
      torch.cuda.empty_cache()


    
  return p_model, q_model


In [ ]:
args = TrainingArguments(
    output_dir="dense_retireval",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=2,
    weight_decay=0.01
)

In [ ]:
# load pre-trained model on cuda (if available)
p_encoder = BertEncoder.from_pretrained(model_checkpoint)
q_encoder = BertEncoder.from_pretrained(model_checkpoint)

if torch.cuda.is_available():
  p_encoder.cuda()
  q_encoder.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertEncoder: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertEncoder: ['cls.predictions.bias',

Train function 정의 후, 두개의 encoder fine-tuning 하기


In [ ]:
p_encoder, q_encoder = train(args, num_neg, train_dataset, p_encoder, q_encoder)

Iteration:   0%|          | 0/10 [00:00<?, ?it/s]

tensor(6.2184, device='cuda:0', grad_fn=<NllLossBackward>)



Iteration:  10%|█         | 1/10 [00:01<00:09,  1.11s/it]

tensor(2.2299, device='cuda:0', grad_fn=<NllLossBackward>)



Iteration:  20%|██        | 2/10 [00:02<00:08,  1.05s/it]

...


Iteration:  80%|████████  | 8/10 [00:07<00:01,  1.08it/s]

tensor(0.0029, device='cuda:0', grad_fn=<NllLossBackward>)



Iteration:  90%|█████████ | 9/10 [00:08<00:00,  1.08it/s]

tensor(0.0035, device='cuda:0', grad_fn=<NllLossBackward>)



Epoch: 100%|██████████| 2/2 [00:18<00:00,  9.38s/it]


## Passage retrieval 준비하기

Search corpus: KorQuAD validation context

In [ ]:
search_corpus = list(set([example['context'] for example in dataset['validation']]))
len(search_corpus)

960

Passage encoder를 활용하여 passage dense embedding 생성

In [ ]:
eval_batch_size = 8

# Construt dataloader
valid_p_seqs = tokenizer(search_corpus, padding="max_length", truncation=True, return_tensors='pt')
valid_dataset = TensorDataset(valid_p_seqs['input_ids'], valid_p_seqs['attention_mask'], valid_p_seqs['token_type_ids'])
valid_sampler = SequentialSampler(valid_dataset)
valid_dataloader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=eval_batch_size)

# Inference using the passage encoder to get dense embeddeings
p_embs = []

with torch.no_grad():

  epoch_iterator = tqdm(valid_dataloader, desc="Iteration", position=0, leave=True)
  p_encoder.eval()

  for _, batch in enumerate(epoch_iterator):
    batch = tuple(t.cuda() for t in batch)

    p_inputs = {'input_ids': batch[0],
                'attention_mask': batch[1],
                'token_type_ids': batch[2]
                }
        
    outputs = p_encoder(**p_inputs).to('cpu').numpy()
    p_embs.extend(outputs)

p_embs = np.array(p_embs)
p_embs.shape  # (num_passage, emb_dim)


Iteration: 100%|██████████| 120/120 [00:18<00:00,  6.66it/s]


(960, 768)

Question encoder를 활용해여 quesntion dense embedding 생성

In [ ]:
sample_idx = np.random.choice(range(len(dataset['validation'])), 5)
query = dataset['validation'][sample_idx]['question']
ground_truth = dataset['validation'][sample_idx]['context']

query

['러시아측이 조사단의 요약보고서를 통보한 두 나라는?',
 '김태희가 출연한 영화 중천은 어느 나라에서 올로케이션으로 촬영됐는가?',
 '김대중 비자금 수사를 유보한 이유로 어디를 중심으로 국민적인 저항이 발생할 것을 우려해서인가?',
 '2010년 월드컵에서 상대팀 중에 한국이 제일 어려웠다고 말한 사람은?',
 '전지현이 영화 도둑들에서 맡은 역할의 이름은 무엇인가?']

In [ ]:
valid_q_seqs = tokenizer(query, padding="max_length", truncation=True, return_tensors='pt').to('cuda')

with torch.no_grad():
  q_encoder.eval()
  q_embs = q_encoder(**valid_q_seqs).to('cpu').numpy()

torch.cuda.empty_cache()

q_embs.shape  # (num_query, emb_dim)

(5, 768)

## GPU를 활용하여 passage retrieval 수행하기

GPU에서 exhaustive search 수행

In [ ]:
if torch.cuda.is_available():
  p_embs_cuda = torch.Tensor(p_embs).to('cuda')
  q_embs_cuda = torch.Tensor(q_embs).to('cuda')


In [ ]:
import time
start_time = time.time()

dot_prod_scores = torch.matmul(q_embs_cuda, torch.transpose(p_embs_cuda, 0, 1))

rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()
print(rank)

print("--- %s seconds ---" % (time.time() - start_time))

tensor([[641, 257, 124,  ..., 263, 594,  39],
        [257, 850, 371,  ..., 719, 208,  39],
        [257, 641, 124,  ..., 263, 594,  39],
        [259, 641, 257,  ..., 562,  39, 195],
        [257, 371, 641,  ..., 719, 208,  39]], device='cuda:0')
--- 0.017329692840576172 seconds ---


In [ ]:
k = 5 

for i, q in enumerate(query[:1]):
  print("[Search query]\n", q, "\n")
  print("[Ground truth passage]")
  print(ground_truth[i], "\n")

  r = rank[i]
  for j in range(k):
    print("Top-%d passage with score %.4f" % (j+1, dot_prod_scores[i][r[j]]))
    print(search_corpus[r[j]])
  print('\n')

[Search query]
 러시아측이 조사단의 요약보고서를 통보한 두 나라는? 

[Ground truth passage]
2010년 7월 27일 한겨레, MBC는 이와 같은 내용으로 조사단의 요약보고서를 공개하며, 러시아 측이 이를 미국과 중국 두 나라에만 통보하고 이를 한국정부에는 알리지 않았고 한국은 우방국을 통해 간접적으로 요약본을 전달받았다고 주장했다. 국방부측은 “러시아는 합조단의 조사결과를 상당부분 신뢰했기 때문에 유엔 안보리에서 거부권을 가지고 있음에도 의장성명에 동의한 것”이라며 “일부 언론의 보도는 러시아의 이런 입장과 상당 부분 차이가 있어 신뢰하기 어렵고 검토결과 자료라는 것도 정체불명으로 판단한다”고 밝혔다. 주한 러시아 대사관도 한겨레의 보도와 관련해, 러시아 대사관측은 본국으로부터 조사결과를 받은 것이 없고 아직 조사가 진행 중이라고 밝히며 천안함 보고서를 한국 정부에 전달한 사실이 없다고 밝혔다. 

Top-1 passage with score 13.4898
2010년 7월 27일 한겨레, MBC는 이와 같은 내용으로 조사단의 요약보고서를 공개하며, 러시아 측이 이를 미국과 중국 두 나라에만 통보하고 이를 한국정부에는 알리지 않았고 한국은 우방국을 통해 간접적으로 요약본을 전달받았다고 주장했다. 국방부측은 “러시아는 합조단의 조사결과를 상당부분 신뢰했기 때문에 유엔 안보리에서 거부권을 가지고 있음에도 의장성명에 동의한 것”이라며 “일부 언론의 보도는 러시아의 이런 입장과 상당 부분 차이가 있어 신뢰하기 어렵고 검토결과 자료라는 것도 정체불명으로 판단한다”고 밝혔다. 주한 러시아 대사관도 한겨레의 보도와 관련해, 러시아 대사관측은 본국으로부터 조사결과를 받은 것이 없고 아직 조사가 진행 중이라고 밝히며 천안함 보고서를 한국 정부에 전달한 사실이 없다고 밝혔다.
Top-2 passage with score 13.3146
책임무능력자(제753조, 제754조)가 제3자에게 가해행위를 한 때에는 이들을 감독할 법정의무가 있는 자 또는 감독

## FAISS를 활용하여 CPU에서 passage retrieval 수행하기




FAISS SQ8, IVF 를 활용해서 cpu에서 passage retrieval 실습해보기

In [ ]:
import faiss

num_clusters = 16
niter = 5
k = 5

# 1. Clustering
emb_dim = p_embs.shape[-1]
index_flat = faiss.IndexFlatL2(emb_dim)

clus = faiss.Clustering(emb_dim, num_clusters)
clus.verbose = True
clus.niter = niter
clus.train(p_embs, index_flat)
centroids = faiss.vector_float_to_array(clus.centroids)
centroids = centroids.reshape(num_clusters, emb_dim)

quantizer = faiss.IndexFlatL2(emb_dim)
quantizer.add(centroids)


In [ ]:
# 2. SQ8 + IVF indexer (IndexIVFScalarQuantizer)
indexer = faiss.IndexIVFScalarQuantizer(quantizer, quantizer.d, quantizer.ntotal, faiss.METRIC_L2)
indexer.train(p_embs)
indexer.add(p_embs)

In [ ]:
# 3. Search using indexer

start_time = time.time()
D, I = indexer.search(q_embs, k)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.001241445541381836 seconds ---


In [ ]:
print('=======[Distance]=======')
print(D)
print('\n')
print('=======[Index of Top-5 Passages]=======')
print(I)

=======[Distance]=======
[[39.043495 39.302006 40.974434 41.66381  42.00199 ]
 [14.899777 16.27565  16.931946 17.34979  18.005228]
 [56.824154 57.71828  60.971283 61.241043 61.438232]
 [19.691881 19.789524 20.55651  20.759863 21.426535]
 [23.681763 25.275448 26.739151 27.170156 27.623869]]


=======[Index of Top-5 Passages]=======
[[641 257 124 259 844]
 [257 641 371 844 850]
 [257 641 124 844 259]
 [641 257 259 124 844]
 [257 641 371 844 259]]


In [ ]:
for i, q in enumerate(query[:1]):
  print("[Search query]\n", q, "\n")
  print("[Ground truth passage]")
  print(ground_truth[i], "\n")

  d = D[i]
  i = I[i]
  for j in range(k):
    print("Top-%d passage with distance %.4f" % (j+1, d[j]))
    print(search_corpus[i[j]])
  print('\n')

[Search query]
 러시아측이 조사단의 요약보고서를 통보한 두 나라는? 

[Ground truth passage]
2010년 7월 27일 한겨레, MBC는 이와 같은 내용으로 조사단의 요약보고서를 공개하며, 러시아 측이 이를 미국과 중국 두 나라에만 통보하고 이를 한국정부에는 알리지 않았고 한국은 우방국을 통해 간접적으로 요약본을 전달받았다고 주장했다. 국방부측은 “러시아는 합조단의 조사결과를 상당부분 신뢰했기 때문에 유엔 안보리에서 거부권을 가지고 있음에도 의장성명에 동의한 것”이라며 “일부 언론의 보도는 러시아의 이런 입장과 상당 부분 차이가 있어 신뢰하기 어렵고 검토결과 자료라는 것도 정체불명으로 판단한다”고 밝혔다. 주한 러시아 대사관도 한겨레의 보도와 관련해, 러시아 대사관측은 본국으로부터 조사결과를 받은 것이 없고 아직 조사가 진행 중이라고 밝히며 천안함 보고서를 한국 정부에 전달한 사실이 없다고 밝혔다. 

Top-1 passage with distance 39.0435
2010년 7월 27일 한겨레, MBC는 이와 같은 내용으로 조사단의 요약보고서를 공개하며, 러시아 측이 이를 미국과 중국 두 나라에만 통보하고 이를 한국정부에는 알리지 않았고 한국은 우방국을 통해 간접적으로 요약본을 전달받았다고 주장했다. 국방부측은 “러시아는 합조단의 조사결과를 상당부분 신뢰했기 때문에 유엔 안보리에서 거부권을 가지고 있음에도 의장성명에 동의한 것”이라며 “일부 언론의 보도는 러시아의 이런 입장과 상당 부분 차이가 있어 신뢰하기 어렵고 검토결과 자료라는 것도 정체불명으로 판단한다”고 밝혔다. 주한 러시아 대사관도 한겨레의 보도와 관련해, 러시아 대사관측은 본국으로부터 조사결과를 받은 것이 없고 아직 조사가 진행 중이라고 밝히며 천안함 보고서를 한국 정부에 전달한 사실이 없다고 밝혔다.
Top-2 passage with distance 39.3020
책임무능력자(제753조, 제754조)가 제3자에게 가해행위를 한 때에는 이들을 감독할 법정의무가 있는 자